In [1]:
import tensorflow as tf
import numpy as np

#import data
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('MNIST_data', one_hot=True)


Extracting MNIST_data\train-images-idx3-ubyte.gz
Extracting MNIST_data\train-labels-idx1-ubyte.gz
Extracting MNIST_data\t10k-images-idx3-ubyte.gz
Extracting MNIST_data\t10k-labels-idx1-ubyte.gz


In [2]:
# defining input and output placeholders
with tf.name_scope('input_values'):
    x = tf.placeholder(tf.float32,shape=[None, 784],name='Input_data')
    y_ = tf.placeholder(tf.float32, shape=[None,10], name='Output_labels')

In [3]:
with tf.name_scope('input_reshape'):
    x_reshape = tf.reshape(x,[-1,28,28,1])
    tf.summary.image('input',x_reshape,10)

In [4]:
# initialize weight and bias variables by defining  functions

def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev=0.15)
    return tf.Variable(initial)

def bias_variable(shape):
    initial =tf.constant(0.1,shape=shape)
    return tf.Variable(initial)

In [5]:
# define different variables summaries with the following function

def variable_summaries(var):
    with tf.name_scope('summaries'):
        mean = tf.reduce_mean(var)
        tf.summary.scalar('mean', mean)
        
        with tf.name_scope('variance'):
            variance= tf.reduce_mean(tf.square(var-mean))
        tf.summary.scalar('variance', variance)   
        tf.summary.scalar('max',tf.reduce_max(var))
        tf.summary.scalar('min', tf.reduce_min(var))
        tf.summary.histogram('histogtam', var)
    

In [6]:
# create a simple neural network layer that can be reused 

def nn_layer(input_tensor, input_dim, output_dim, layer_name, act=tf.nn.relu):
    with tf.name_scope(layer_name):
        with tf.name_scope('weights'):
            weights = weight_variable([input_dim, output_dim])
            variable_summaries(weights)
        with tf.name_scope('biases'):
            biases = bias_variable([output_dim])
            variable_summaries(biases)
        with tf.name_scope('Wx_plus_b'):
            preactivate = tf.matmul(input_tensor,weights)+ biases
            tf.summary.histogram('pre_activation', preactivate)
        activations = act(preactivate, name='activation')
        tf.summary.histogram('activations', activations)
        return activations
        

In [7]:
hidden1 = nn_layer(x,784,500,'layer1')

In [8]:
with tf.name_scope('dropout'):
    keep_prob = tf.placeholder(tf.float32)
    tf.summary.scalar('dropout_keep_prob', keep_prob)
    dropped = tf.nn.dropout(hidden1, keep_prob)

In [9]:
y = nn_layer(dropped,500,10,'layer2',act=tf.identity)

In [10]:
with tf.name_scope('cross_entropy'):
    diff = tf.nn.softmax_cross_entropy_with_logits(labels=y_, logits=y)
    with tf.name_scope('total'):
        cross_entropy = tf.reduce_mean(diff)
tf.summary.scalar('cross_entropy', cross_entropy)

with tf.name_scope('train'):
    train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)
    

In [11]:
with tf.name_scope('accuracy'):
    with tf.name_scope('correct_prediction'):
        correct_prediction = tf.equal(tf.argmax(y,1),tf.argmax(y_,1))
    with tf.name_scope('accuracy'):
        accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
tf.summary.scalar('accuracy', accuracy)

merged = tf.summary.merge_all()
train_writer = tf.summary.FileWriter("./train_logs",graph=tf.get_default_graph())
test_writer = tf.summary.FileWriter("./test_logs",graph=tf.get_default_graph())

init =tf.global_variables_initializer()

In [12]:
with tf.Session() as sess:
    sess.run(init)
    
    def feed_dict(train):
        if train:
            xs, ys = mnist.train.next_batch(50)
            k=0.5
        else:
            xs, ys =mnist.test.images, mnist.test.labels
            k = 0.5
        return {x:xs, y_:ys, keep_prob:k}
    
    for i in range(2000):
        if i% 10 ==0:
            summary, acc = sess.run([merged, accuracy], feed_dict=feed_dict(False))
            test_writer.add_summary(summary,i)
            print('Accuracy at step %s: %s' %(i,acc))
        else: 
            if i % 100 == 99:  # Record execution stats
                run_options = tf.RunOptions(trace_level=tf.RunOptions.FULL_TRACE)
                run_metadata = tf.RunMetadata()
                summary, _ = sess.run([merged, train_step],
                                      feed_dict=feed_dict(True),
                                      options=run_options,run_metadata=run_metadata)
                train_writer.add_run_metadata(run_metadata, 'step%03d' % i)
                train_writer.add_summary(summary, i)
                print('Adding run metadata for', i)
            else:  # Record a summary
                summary, _ = sess.run([merged, train_step], feed_dict=feed_dict(True))
                train_writer.add_summary(summary, i)
    train_writer.close()
    test_writer.close()
    
    
    

Accuracy at step 0: 0.1213
Accuracy at step 10: 0.1426
Accuracy at step 20: 0.1499
Accuracy at step 30: 0.1697
Accuracy at step 40: 0.1902
Accuracy at step 50: 0.2141
Accuracy at step 60: 0.226
Accuracy at step 70: 0.2441
Accuracy at step 80: 0.2626
Accuracy at step 90: 0.2881
Adding run metadata for 99
Accuracy at step 100: 0.3184
Accuracy at step 110: 0.3328
Accuracy at step 120: 0.3536
Accuracy at step 130: 0.3746
Accuracy at step 140: 0.393
Accuracy at step 150: 0.4107
Accuracy at step 160: 0.4312
Accuracy at step 170: 0.4357
Accuracy at step 180: 0.4542
Accuracy at step 190: 0.466
Adding run metadata for 199
Accuracy at step 200: 0.481
Accuracy at step 210: 0.4946
Accuracy at step 220: 0.5042
Accuracy at step 230: 0.5186
Accuracy at step 240: 0.5301
Accuracy at step 250: 0.5293
Accuracy at step 260: 0.5505
Accuracy at step 270: 0.5614
Accuracy at step 280: 0.5716
Accuracy at step 290: 0.5774
Adding run metadata for 299
Accuracy at step 300: 0.5738
Accuracy at step 310: 0.5863
Accu

In [ ]:
dirpath= "./t"
if tf.gfile.Exists(dirpath):
    tf.gfile.DeleteRecursively(dirpath) 